第一步先写一个快速比较字符串，召回的类，使得召回率尽量达到百分之96左右，精度百分之30左右最好
字符串距离函数采取jaccard 计算快
下一步可增加编辑距离、cosine距离 来看一下速度快不快
因为需要召回的速度快，召回的召回率高，因此需要损失掉较高的精度，下一步实体消歧义处理精度的问题

In [1]:
import json
from tqdm import tqdm
import os
from random import choice
from itertools import groupby
import numpy as np 
# import jieba


读入knowledge base的数据，建立倒排索引表等

In [2]:
class KB(object):
    def __init__(self,kb_directory):
        print("start loading kb_data...")
        self.kb_directory = kb_directory
        self.id2kb,self.types,self.predicate = self.get_id2kb()
        self.kb2id = self.get_kb2id()
        self.kb = list(self.kb2id.keys())
        self.id = list(self.id2kb.keys())
        self.print_info()

    def print_info(self):
        print("KB DATA INFORMATION")
        print("TOKEN SIZE:{}".format(self.get_token_size()))
        print("ID SIZE:{}".format(len(self)))
        print("TYPE SIZE:{}".format(len(self.types)))
        print("PREDICATE SIZE:{}".format(len(self.predicate)))
        
    def get_id2kb(self):
        print("construct id2kb dict...")
        id2kb = {}
        kbtype = set()
        predicate = set()
        multi_type = []
        with open(self.kb_directory) as f:
            for l in tqdm(f):
                tmp = json.loads(l)
                subject_id = tmp['subject_id']
                subject_alias = list(set([tmp['subject']] + tmp.get('alias', [])))
                subject_alias = [alias.lower() for alias in subject_alias]
                subject_type = [i.lower() for i in tmp['type']]
                kbtype.update(subject_type)
                try:
                    assert(len(tmp['type'])==1)
                except AssertionError:
                    multi_type.append(tmp['type'])
                subject_data = {}
                for i in tmp['data']:
                    predicate.add(i['predicate'].lower())
                    subject_data[i['predicate'].lower()] = i['object'].lower()
                if subject_data:
                    id2kb[subject_id] = {'alias': subject_alias, 'data': subject_data,'type':subject_type}
#         print(multi_type)
        return id2kb,kbtype,predicate

    def get_kb2id(self):
        print("construct kb2id dict...")
        kb2id = {}
        for i,j in self.id2kb.items():
            for k in j['alias']:
                if k not in kb2id:
                    kb2id[k] = []
                kb2id[k].append(i)
        return kb2id
    
    def __len__(self):
        return len(self.id2kb)
    
    def get_token_size(self):
        return len(self.kb)

In [3]:
kb_data = KB('./ccks2019_el/kb_data')

4548it [00:00, 22381.57it/s]

start loading kb_data...
construct id2kb dict...


399252it [00:18, 21357.40it/s]


construct kb2id dict...
KB DATA INFORMATION
TOKEN SIZE:303375
ID SIZE:399233
TYPE SIZE:51
PREDICATE SIZE:41841


读入训练数据，验证数据，测试数据。测试数据不做错误分析！！！

In [4]:
def load_data(data_directory):
    with open(data_directory) as f:
        train_data,dev_data,test_data = json.load(f)
    print('traindata size:',len(train_data))
    print('devdata size:',len(dev_data))
    print('testdata size',len(test_data))
    return train_data,dev_data,test_data

In [5]:
train_data,dev_data,test_data = load_data('./data/all_data.json')
print(train_data[0])

traindata size: 84262
devdata size: 851
testdata size 852
{'text_id': '81967', 'text': '电影《淘金岁月》下载 急求', 'mention_data': [['电影', 0, '148097'], ['淘金岁月', 3, '50357']]}


In [6]:
l = [len(i) for i in kb_data.kb]
print(max(l))
max_l = max(l)
for i in kb_data.kb:
    if len(i)==max_l:
        print(i)

108
apis cerana cerana fabricius apis cerana cerana fabricius apis cerana cerana fabricius apis cerana fabricius


生成候选集，候选集按offset排序,把字符串相似度放到结果中，并按照相似程度排序

In [27]:
from tsim import TopSim
import jieba
import thulac

In [8]:
" ".join(jieba.cut('apis cerana cerana fabricius apis cerana cerana fabricius apis cerana cerana fabricius apis cerana fabricius'))

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 1.070 seconds.
Prefix dict has been built succesfully.


'apis   cerana   cerana   fabricius   apis   cerana   cerana   fabricius   apis   cerana   cerana   fabricius   apis   cerana   fabricius'

In [9]:
def check_chinese(check_str):
    '''
判断分词后的字符串是否为中文
    '''
    for ch in check_str:
        if u'\u4e00' <= ch <= u'\u9fff':
            return True
        else:
            return False

In [10]:
print(check_chinese('青春'))
print(check_chinese('youth'))

True
False


In [28]:

thu = thulac.thulac(seg_only=True)
def build_ngram_c(cut_words):
    tmp_off = []
    tmp = []
    last = 0
    for i in range(len(cut_words)):
        w = cut_words[i]
        if check_chinese(w) and len(w)>2:
            _ = thu.cut(w)
            if len(_) > 1:
                _ = [j[0] for j in _]
                for j in _:
                    tmp_off.append((last,last+len(j)))
                    last = tmp_off[-1][1]
            else:
                last += len(w)
        else:
            last = last + len(w)
    return tmp,tmp_off

Model loaded succeed


In [29]:
a,b = build_ngram_c(['youth','青春'])
print(a)
print(b)

[]
[]


In [13]:
dev_x = [i['text'] for i in dev_data]
cut_data = [list(jieba.cut(i)) for i in dev_x]
cut_data_l = [[len(j) for j in i] for i in cut_data]
print(dev_x[0])
en = []
en_off = []
for i in dev_data:
    tmp = []
    _ = []
    for j in i['mention_data']:
        tmp.append(j[0])
        _.append((str(j[1]),j[0]))
    en.append(tmp)
    en_off.append(_)
print(en[0])
print(en_off[0])
print(cut_data[0])
print(cut_data_l[0])

《铁梨花》,结局让人有着说不出的失落。 - 铁梨花
['铁梨花', '结局', '失落', '铁梨花']
[('1', '铁梨花'), ('6', '结局'), ('16', '失落'), ('22', '铁梨花')]
['《', '铁', '梨花', '》', ',', '结局', '让', '人', '有着', '说不出', '的', '失落', '。', ' ', '-', ' ', '铁', '梨花']
[1, 1, 2, 1, 1, 2, 1, 1, 2, 3, 1, 2, 1, 1, 1, 1, 1, 2]


In [ ]:
max_cl = 0
avg = []
for i in range(len(cut_data_l)):
    l = cut_data_l[i]
    avg.append(sum(l)/len(l))
    if max(l) > max_cl:
        max_cl = max(l)
        tmp = i
print(max_cl)
print(sum(avg)/len(avg))
print(cut_data[tmp])
print(cut_data_l[tmp])
# print(avg)

In [38]:
class ngram_search(object):
    
    def __init__(self,data,kb,ngram = 10,similarity = 0.5,k=1,e=0.7,simFunc="tversky"): 
        '''
        ngram是如何合并词语,
        similarity是搜到的字符串和原本字符串的相似度，
        k是选取几个相似词，
        e是平衡分母是更多的依赖query的长度还是entity的长度
        '''

        self.n = ngram
#         self.ngram_c = ngram_c
        self.similarity = similarity
        self.data = data
        self.kb = kb
        self.k = k
        self.e = e
        self.simFunc = simFunc
        self.thu = thulac.thulac(seg_only=True)
        self.cut_data,self.offset = self.cut_words()
        self.ts = TopSim(self.kb)
        self.candidates = self.get_candidates()
        self.cand_name,self.cand_off,self.cand_with_off,self.cand_with_offend,self.ment_data = self.get_candidates_name()
    
    
    def build_ngramc(self,cut_words):
        tmp_off = []
        tmp = []
        last = 0
        for i in range(len(cut_words)):
            w = cut_words[i]
            if check_chinese(w) and len(w)>2:
                _ = self.thu.cut(w)
                if len(_) > 1:
                    _ = [j[0] for j in _]
                    for j in _:
                        tmp_off.append((last,last+len(j)))
                        last = tmp_off[-1][1]
                else:
                    last += len(w)
            else:
                last = last + len(w)
                
#         tmp_off.append((0,len(tmp[0])))
#         n = len(tmp)
#         for i in range(1,len(tmp)):
#             tmp_off.append((tmp_off[-1][1],tmp_off[-1][1]+len(tmp[i])))
        
#         for j in range(2,self.ngram_c+1):
#             for i in range(j-1,n):
#                 tmp.append(''.join(tmp[i-j+1:i+1]))
#                 tmp_off.append((tmp_off[i-j+1][0],tmp_off[i][1]))
        
        return tmp,tmp_off
    
    def cut_words(self):
        '''
        切词生成ngram,加上字特征
        '''
        print('starting build ngram list')
        print('ngram',self.n)
#         print('ngram_c',self.ngram_c)
        result = []
        offset = []
#         thu = thulac.thulac(seg_only=True)
        for d in tqdm(self.data):
            tmp = list(jieba.cut(d))
#             _ = thu.cut(d)
#             tmp = [i[0] for i in _]
#             tmp = tmp.split(' ')
            n = len(tmp)
            tmp_off = []
            tmp_off.append((0,len(tmp[0])))
            for i in range(1,len(tmp)):
                tmp_off.append((tmp_off[-1][1],tmp_off[-1][1]+len(tmp[i])))
            for j in range(2,self.n+1):
                for i in range(j-1,n):
                    tmp.append(''.join(tmp[i-j+1:i+1]))
                    tmp_off.append((tmp_off[i-j+1][0],tmp_off[i][1]))
            tmp.append(d)
            tmp_off.append((0,len(d)))
            c_tmp,c_tmp_off = self.build_ngramc(tmp)
            tmp.extend(c_tmp)
            tmp_off.extend(c_tmp_off)
#             for i in range(len(c_tmp_off)):
#                 if c_tmp_off[i] not in tmp_off:
#                     tmp.append(c_tmp[i])
#                     tmp_off.append(c_tmp_off[i])
            result.append(tmp)
            offset.append(tmp_off)
        return result,offset
    
    def get_candidates(self):
        '''
        从数据库中选取topk
        '''
#         self.similarity = similarity
        print('starting build candidates list')
        print('similarity:',self.similarity)
        candidates = []
        for dt in tqdm(self.cut_data):
            ts_result = []
            for i in dt:
                tmp = self.ts.search(i,k = self.k,e = self.e,worstSim = self.similarity,simFunc=self.simFunc)
                if tmp:
#                 if tmp and tmp[0][0] > self.similarity:
                    ts_result.append(tmp)
                else:
                    ts_result.append([])
            candidates.append(ts_result)
        return candidates
    
    def get_candidates_name(self):
        '''
        召回结果，短字符串的offset更优先
        [[ment],[cand],[sim],[jump]]，一一对应
        '''
        print('starting get candidates name and offset')
        cand_name = []
        ment_data = []
        cand_offset = []
        cand_with_off = []
        cand_with_offend = []
        for i in tqdm(range(len(self.candidates))):
            cand = []
            off = []
            ment = []
            c_o_s = {}
            c_o_e = {}
            for j in range(len(self.candidates[i])):    
                if self.candidates[i][j]:
                    token = self.kb[self.candidates[i][j][0][1][0]]
                    sim = self.candidates[i][j][0][0]
                    gram = self.cut_data[i][j]
                    token_off = self.offset[i][j]
                    begin = str(token_off[0])
                    end = str(token_off[1]) 
                    if end in c_o_e:
                        if token in c_o_e[end]:
                            continue
                    else:
                        c_o_e[end] = set()
                    if begin in c_o_s:
                        if token in c_o_s[begin][1]:
                            continue
                    else:
                        c_o_s[begin] = [[],[],[],[]]
                    cand.append(token)
                    off.append(token_off)
                    ment.append(gram)
                    c_o_s[begin][0].append(gram)
                    c_o_s[begin][1].append(token)
                    c_o_s[begin][2].append(sim)
                    c_o_s[begin][3].append(token_off[1])
#                     c_o_s[begin].append((token_off[1],token))
                    c_o_e[end].add(token)
            cand_name.append(cand)
            cand_offset.append(off)
            ment_data.append(ment)
            cand_with_off.append(c_o_s)
            cand_with_offend.append(c_o_e)
        return cand_name,cand_offset,cand_with_off,cand_with_offend,ment_data

In [39]:
ns = ngram_search(dev_x,kb_data.kb)

  1%|          | 6/851 [00:00<00:52, 16.25it/s]

Model loaded succeed
starting build ngram list
ngram 10


100%|██████████| 851/851 [00:15<00:00, 55.03it/s]


starting building grammap...
gram map infomation:
keys number: 266779


  0%|          | 2/851 [00:00<01:18, 10.81it/s]

starting build candidates list
similarity: 0.5


100%|██████████| 851/851 [00:00<00:00, 7153.89it/s]

starting get candidates name and offset


In [40]:
print(ns.cut_data[0])
print(ns.offset[0])

['《', '铁', '梨花', '》', ',', '结局', '让', '人', '有着', '说不出', '的', '失落', '。', ' ', '-', ' ', '铁', '梨花', '《铁', '铁梨花', '梨花》', '》,', ',结局', '结局让', '让人', '人有着', '有着说不出', '说不出的', '的失落', '失落。', '。 ', ' -', '- ', ' 铁', '铁梨花', '《铁梨花', '铁梨花》', '梨花》,', '》,结局', ',结局让', '结局让人', '让人有着', '人有着说不出', '有着说不出的', '说不出的失落', '的失落。', '失落。 ', '。 -', ' - ', '- 铁', ' 铁梨花', '《铁梨花》', '铁梨花》,', '梨花》,结局', '》,结局让', ',结局让人', '结局让人有着', '让人有着说不出', '人有着说不出的', '有着说不出的失落', '说不出的失落。', '的失落。 ', '失落。 -', '。 - ', ' - 铁', '- 铁梨花', '《铁梨花》,', '铁梨花》,结局', '梨花》,结局让', '》,结局让人', ',结局让人有着', '结局让人有着说不出', '让人有着说不出的', '人有着说不出的失落', '有着说不出的失落。', '说不出的失落。 ', '的失落。 -', '失落。 - ', '。 - 铁', ' - 铁梨花', '《铁梨花》,结局', '铁梨花》,结局让', '梨花》,结局让人', '》,结局让人有着', ',结局让人有着说不出', '结局让人有着说不出的', '让人有着说不出的失落', '人有着说不出的失落。', '有着说不出的失落。 ', '说不出的失落。 -', '的失落。 - ', '失落。 - 铁', '。 - 铁梨花', '《铁梨花》,结局让', '铁梨花》,结局让人', '梨花》,结局让人有着', '》,结局让人有着说不出', ',结局让人有着说不出的', '结局让人有着说不出的失落', '让人有着说不出的失落。', '人有着说不出的失落。 ', '有着说不出的失落。 -', '说不出的失落。 - ', '的失落。 - 铁', '失落。 - 铁梨花', '《铁梨花》,结局让人', '铁梨花》,结局让

In [41]:
print(dev_x[0][22:25])

铁梨花


In [42]:
print(ns.candidates[0])

[[], [(1.0, [44529])], [(1.0, [4657])], [], [], [(1.0, [10911])], [], [(1.0, [48458])], [], [(0.5555555555555556, [76677])], [(1.0, [35640])], [(1.0, [108397])], [], [], [], [], [(1.0, [44529])], [(1.0, [4657])], [], [(1.0, [4605])], [(0.6060606060606061, [4657])], [], [(0.6060606060606061, [10911])], [(0.6060606060606061, [10911])], [], [], [], [(0.625, [84779])], [(0.6060606060606061, [108397])], [(0.6060606060606061, [108397])], [], [], [], [], [(1.0, [4605])], [(0.6976744186046512, [4605])], [(0.6976744186046512, [4605])], [(0.5555555555555556, [4657])], [(0.5555555555555556, [10911])], [], [(0.5555555555555556, [10911])], [], [], [], [(0.5714285714285714, [84779])], [], [(0.5555555555555556, [108397])], [], [], [], [(0.6976744186046512, [4605])], [(0.5660377358490565, [45132])], [(0.6521739130434783, [4605])], [], [], [], [], [], [], [], [(0.547945205479452, [84779])], [], [(0.5128205128205129, [108397])], [], [], [(0.6521739130434783, [4605])], [], [(0.5769230769230769, [4605])],

In [43]:
print(ns.cand_with_off[0])
print(ns.cand_with_offend[0])
print(ns.ment_data[0])
print(ns.cand_name[0])
print(ns.cand_off[0])

{'1': [['铁', '铁梨花'], ['铁', '铁梨花'], [1.0, 1.0], [2, 4]], '2': [['梨花'], ['梨花'], [1.0], [4]], '6': [['结局'], ['结局'], [1.0], [8]], '9': [['人'], ['人'], [1.0], [10]], '12': [['说不出', '说不出的'], ['说不出再见', '说不出的快活'], [0.5555555555555556, 0.625], [15, 16]], '15': [['的'], ['的'], [1.0], [16]], '16': [['失落'], ['失落'], [1.0], [18]], '22': [['铁', '铁梨花'], ['铁', '铁梨花'], [1.0, 1.0], [23, 25]], '23': [['梨花'], ['梨花'], [1.0], [25]], '0': [['《铁梨花》'], ['《莲花》'], [0.5660377358490565], [5]]}
{'2': {'铁'}, '4': {'铁梨花', '梨花'}, '8': {'结局'}, '10': {'人'}, '15': {'说不出再见'}, '16': {'的', '说不出的快活'}, '18': {'失落'}, '23': {'铁'}, '25': {'铁梨花', '梨花'}, '5': {'《莲花》'}, '9': set(), '19': set(), '6': set(), '20': set(), '21': set()}
['铁', '梨花', '结局', '人', '说不出', '的', '失落', '铁', '梨花', '铁梨花', '说不出的', '铁梨花', '《铁梨花》']
['铁', '梨花', '结局', '人', '说不出再见', '的', '失落', '铁', '梨花', '铁梨花', '说不出的快活', '铁梨花', '《莲花》']
[(1, 2), (2, 4), (6, 8), (9, 10), (12, 15), (15, 16), (16, 18), (22, 23), (23, 25), (1, 4), (12, 16), (22, 25), (0, 5)]


计算候选集的召回情况

In [44]:
from extratools.mathtools import safediv

def evaluate_cand_off(cand_with_off,ground_truth,ment_data,k=0.6):
    recall = []
    precision = []
    f1 = []
    under_k = []
    error = []
    error_off = []
    
    for i in tqdm(range(len(ground_truth))):
        tp = 0
        for j in ground_truth[i]:
            if j[0] in cand_with_off[i]:
                if j[1] in cand_with_off[i][j[0]][0]:
                    tp += 1
            else:
                if j[1] in ment_data[i]:
                    error_off.append(i)        
        r = tp/len(ground_truth[i])
        if r < k:
            under_k.append(i)
        p = tp/len(cand_with_off[i].keys())
        f = safediv(2*r*p,r+p)
        
        recall.append(r)
        precision.append(p)
        if  f > 1 or np.isnan(f):
            error.append(i)
            f1.append(0)
        else:
            f1.append(f)
            
    av_recl = sum(recall)/len(recall)
    av_pre = sum(precision)/len(precision)
    av_f1 = sum(f1)/len(f1)
    
    print('average recall: {}'.format(av_recl))
    print('average precision: {}'.format(av_pre))
    print('average f1: {}'.format(av_f1))
    print('error number: {}'.format(len(error)))
    print('total number: {}'.format(len(ment_data)))
    print('error rate: {}'.format(len(error)/len(ment_data)))
    print('error off: {}'.format(len(error_off)))
    print('recall under{} : {}'.format(k,len(under_k)))
    return recall, precision,f1,error,error_off,under_k

In [45]:
recall, precision,f1,error,error_off,under_k = evaluate_cand_off(ns.cand_with_off,en_off,ns.ment_data)

100%|██████████| 851/851 [00:00<00:00, 108646.17it/s]

average recall: 0.8468002163654335
average precision: 0.29549812769122286
average f1: 0.41816250183386683
error number: 37
total number: 851
error rate: 0.043478260869565216
error off: 6
recall under0.6 : 127


In [ ]:
print(en_off[0])

In [46]:
for i in error_off:
    print(ns.cand_with_off[i])
    print(en_off[i])
    print(ns.ment_data[i])
    print(ns.cand_off[i])
    print(dev_x[i])
#     print(ns.cut_data[i])
    print()

{'1': [['心魔', '心魔】我有心'], ['心魔', '心'], [1.0, 0.5714285714285714], [3, 7]], '4': [['我', '我有心魔人'], ['我', '我有恋人'], [1.0, 0.5660377358490565], [5, 9]], '5': [['有心'], ['有心人'], [0.5405405405405405], [7]], '7': [['魔人'], ['魔人'], [1.0], [9]], '9': [['有心', '有心魔'], ['有心人', '心魔'], [0.5405405405405405, 0.6060606060606061], [11, 12]], '11': [['魔'], ['魔'], [1.0], [12]], '16': [['要'], ['要'], [1.0], [17]], '17': [['怎样'], ['怎样'], [1.0], [19]], '0': [['【心魔】'], ['【】'], [0.5555555555555556], [4]]}
[('1', '心魔'), ('6', '心魔'), ('10', '心魔')]
['心魔', '我', '有心', '魔人', '有心', '魔', '要', '怎样', '有心魔', '【心魔】', '我有心魔人', '心魔】我有心']
[(1, 3), (4, 5), (5, 7), (7, 9), (9, 11), (11, 12), (16, 17), (17, 19), (9, 12), (0, 4), (4, 9), (1, 7)]
【心魔】我有心魔人有心魔吗。。?要怎样

{'1': [['心魔', '心魔】我有心'], ['心魔', '心'], [1.0, 0.5714285714285714], [3, 7]], '4': [['我', '我有心魔人'], ['我', '我有恋人'], [1.0, 0.5660377358490565], [5, 9]], '5': [['有心'], ['有心人'], [0.5405405405405405], [7]], '7': [['魔人'], ['魔人'], [1.0], [9]], '9': [['有心', '有心魔'], ['有心人', '心魔'], [0.

In [48]:
for i in under_k:
    print(ns.cand_with_off[i])
    print(en_off[i])
    print(ns.ment_data[i])
    print(ns.cand_off[i])
    print(dev_data[i])
    print()

{'1': [['我', '我曾这样'], ['我', '我曾这样寂寞生活'], [1.0, 0.5128205128205129], [2, 5]], '2': [['曾', '曾这样寂寞生活:'], ['曾', '我曾这样寂寞生活'], [1.0, 0.6666666666666666], [3, 10]], '5': [['寂寞', '寂寞生活:辛波斯卡诗选'], ['寂寞', '我曾这样寂寞生活：辛波斯卡诗选2'], [1.0, 0.5161290322580645], [7, 16]], '7': [['生活', '生活:辛波斯卡诗选2'], ['生活', '我曾这样寂寞生活：辛波斯卡诗选2'], [1.0, 0.5263157894736842], [9, 17]], '10': [['辛', '辛波斯卡'], ['辛', '辛·阿斯卡'], [1.0, 0.5263157894736843], [11, 14]], '11': [['波斯'], ['波斯'], [1.0], [13]], '13': [['卡'], ['卡'], [1.0], [14]], '14': [['诗选'], ['诗选'], [1.0], [16]], '16': [['2'], ['2'], [1.0], [17]], '18': [['的'], ['的'], [1.0], [19]], '19': [['笔记'], ['笔记'], [1.0], [21]], '22': [['种种'], ['种'], [0.8695652173913044], [24]], '24': [['可能'], ['可能'], [1.0], [26]], '3': [['这样寂寞生活', '这样寂寞生活:辛波斯卡'], ['我曾这样寂寞生活', '我曾这样寂寞生活：辛波斯卡诗选2'], [0.7142857142857143, 0.5161290322580645], [9, 14]], '0': [['《我曾这样寂寞', '《我曾这样寂寞生活:辛波斯卡诗选2》的笔记-种种可能'], ['我曾这样寂寞生活', '我曾这样寂寞生活：辛波斯卡诗选2'], [0.574712643678161, 0.65], [7, 26]]}
[('1', '我曾这样寂寞生活:辛波斯卡诗选2'), ('19', '

In [ ]:
print(kb_data.id2kb['142956'])

In [ ]:
ts = TopSim(kb_data.kb)

In [ ]:
print(under_k[-1])

In [ ]:
s_r = ts.search('孙红',k=1,e = 0.7,worstSim = 0.5,simFunc="tversky")
print(s_r)

In [ ]:
for i in s_r[0][1]:
    print(kb_data.kb[i])

In [ ]:
thu1 = thulac.thulac(seg_only=True)  #默认模式
text = thu1.cut("《好先生》孙红床上难忍 叫声不断")  #进行一句话分词
print(text)

In [ ]:
import jieba
print(list(jieba.cut("《好先生》孙红床上难忍 叫声不断")))

In [ ]:
for i in error:
    print(ns.cand_with_off[i])
    print(en_off[i])
    print(ns.ment_data[i])
    print(ns.offset[i])
    print(ns.cut_data[i])
    print(dev_data[i])
    print()

In [ ]:
import pkuseg

In [ ]:
seg = pkuseg.pkuseg()           # 以默认配置加载模型
text = seg.cut("《好先生》孙红床上难忍 叫声不断")  # 进行分词
print(text)

In [ ]:
import jieba
s = '有首歌一男一女唱的。里面有“我会一直陪在你身旁”是...'
seg_list = jieba.cut(s, cut_all=True)
print("Full Mode: " + "/ ".join(seg_list))  # 全模式

seg_list = jieba.cut(s, cut_all=False)
print("Default Mode: " + "/ ".join(seg_list))  # 精确模式

seg_list = jieba.cut(s)  # 默认是精确模式
print(", ".join(seg_list))

seg_list = jieba.cut_for_search(s)  # 搜索引擎模式
print(", ".join(seg_list))